In [15]:
print("A)")

A)


In [16]:
price = (8022+3300)+24444+999+2000+2400+3107+322+1089+1733
print(price)

47416


# overview 
In this notebook, we will implementing the K-Nearest Neighbors(KNN) algorithm from scratch. KNN is a surpervised macine learning algorithm that can be used for both classification and regression tasks. It is a non-parametric method that makes predictions based on the simillarity of new data points to the labeled data points in the training set.
 
The main objective of this notebook is to understand the inner workings of the KNN algorithm and how it can be implemented without relying on existing libraries or frameworks. By building the algorithm from scratch, we will gain a deeper understanding of its underlying principles and mechanics.

# kNNアルゴリズムとは
k近傍法(k-NearestNeighbor)アルゴリズムは、近接性を使用して個々のデータポイントのグループ化に関する分類または予測を行うノンパラメトリックな教師あり学習分類器である。
kNNアルゴリズムは回帰問題にも分類問題にも利用できるが、通常は分類アルゴリズムとして使用される。
基本的なアイデアは多数決である。カテゴリ数が2つの場合には「多数決」カテゴリ数がそれ以上の場合には「相対多数」と呼ばれる。あるデータポイントからk番目までに近いポイントのデータラベルの相対多数によって、そのデータポイントのラベルを決定する。
kNNアルゴリズムは「遅延学習」と呼ばれる分類に属し、トレーニングを実行するのではなく、トレーニング・データセットの保存のみを実行する、

# kNNの計算：距離メトリクス
kNN


ノンパラメトリック：与えられた母集団が何らかの分布に従っている前提がない
パラメトリック：与えられた母集団が何らかの分布に従っている前提がある（だいたい正規分布）
回帰問題：入力データから数値の予測を行う問題。この場合与えられたデータポイントがどのグループに属するかを予測する
分類問題：与えられたデータのデータポイントがそれぞれどこに属するか
遅延学習：計算が必要になるまで計算を行なわない　kNNでは計算の方法のみを保管し、計算の実行は分類/予測が行われているときに行う

# imports

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns


In [18]:
#data analysis